In [ ]:
inpath = ""
outpath = ""

# XAS Notebook
The XAS Notebook autoprocessor will run automatically on every scan performed that contains the NXxas sub-entry.

The analysis runs in 4 steps:
1. Read the NeXus file using [HdfMap](https://github.com/DiamondLightSource/hdfmap) to find the metadata and scanned values using field names expected for beamlines i06-1 or i10-1. Plot the data extracted.
2. Read and analyse the XAS scan data using [mmg_toolbox.xas](https://github.com/DiamondLightSource/mmg_toolbox/tree/main/mmg_toolbox/xas) functions, this includes creating an xas_scan object containing Spectra objects - providing a framework for tracked analysis functions. Basic normalisation steps are performed and a processed NXxas NeXus file is produced.
3. Recent scans in the same directory are compared with this one, if any similar scans are found they are plotted for comparison.
4. If the similar scans have the opposite polarisation, automatic subtraction is performed to calculate XMCD spectra, which is output as a different processed NXxas NeXus file.


### Note
**mmg_toolbox** is in active development and likely to change in the future, therefore please don't rely to heavily on the functions at this point. You can see how everything works at the [repository](https://github.com/DiamondLightSource/mmg_toolbox). If you have any comments, please do get in touch!

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

from mmg_toolbox import xas, data_file_reader, module_info

print(module_info())


In [ ]:
# Load data from NeXus file using HdfMap
# Works across multiple NeXus file types - i06, i10 etc
scan = data_file_reader(inpath)

with scan.map.load_hdf() as nxs:
    def rd(expr, default=''):
        return scan.map.format_hdf(nxs, expr, default=default)

    # currently accounts for i06-1 and i10-1 metadata
    metadata = {
        "scan": rd('{filename}'),
        "cmd": rd('{(cmd|user_command|scan_command)}'),
        "title": rd('{title}', os.path.basename(inpath)),
        "endstation": rd('{end_station}', 'unknown'),
        "sample": rd('{sample_name}', ''),
        "energy": rd('{mean((fastEnergy|pgm_energy|energye|energyh)):.2f} eV'),
        "pol": rd('{polarisation?("lh")}'),
        "height": rd('{(em_y|hfm_y):.2f}', 0),
        "pitch": rd('{(em_pitch|hfm_pitch):.2f}', 0),
        "temperature": rd('{(T_sample|sample_temperature|lakeshore336_cryostat|lakeshore336_sample|itc3_device_sensor_temp?(300)):.2f} K'),
        "field": rd('{(field_z|sample_field|magnet_field|ips_demand_field?(0)):.2f} T'),
    }
    ENERGY = '(fastEnergy|pgm_energy|energye|energyh)'
    MONITOR = '(i0|C2|ca62sr|mcs16_data|mcse16_data|mcsh16_data)'
    TEY = '(tey|C1|ca61sr|mcs17_data|mcse17_data|mcsh17_data)'
    TFY = '(fdu|C3|ca63sr|mcs18_data|mcse18_data|mcsh18_data|mcsd18_data)'

    print('Scan data paths:')
    print('energy: ', scan.map.eval(nxs, '_' + ENERGY))
    print('monitor: ', scan.map.eval(nxs, '_' + MONITOR))
    print('tey: ', scan.map.eval(nxs, '_' + TEY))
    print('tfy: ', scan.map.eval(nxs, '_' + TFY))

    energy = scan.map.eval(nxs, ENERGY)
    monitor = scan.map.eval(nxs, MONITOR, default=1.0)
    tey = scan.map.eval(nxs, TEY, default=np.ones(scan.map.scannables_shape())) / monitor
    tfy = scan.map.eval(nxs, TFY, default=np.ones(scan.map.scannables_shape())) / monitor

print('\nMetadata:')
print('\n'.join(f"{n:12}: {d}" for n, d in metadata.items()))

title = "{endstation} {sample} {scan}\nE = {energy}, pol = {pol}, T = {temperature}, B = {field}".format(**metadata)
print('\ntitle: ', title)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 4))
fig.suptitle(title)

ax[0].plot(energy, tey, '-', label=f"{metadata['pol']} TEY")
ax[1].plot(energy, tfy, '-', label=f"{metadata['pol']} TFY")

ax[0].set_xlabel('Energy [eV]')
ax[0].set_ylabel('signal / monitor')
ax[0].legend()

ax[1].set_xlabel('Energy [eV]')
ax[1].set_ylabel('signal / monitor')
ax[1].legend()

In [ ]:
# Get absorption edge from energy
print(f"Absorption edges between: {energy.min()}, {energy.max()} eV")
available_edges = xas.xray_edges_in_range(energy.min(), energy.max(), search_edges=None)
print('\n'.join(f"{en} eV : {lab}" for en, lab in available_edges))

# search L edges only, returns single element edge or set
available_l_edges = xas.xray_edges_in_range(energy.min(), energy.max())
edge_label = ' '.join(xas.energy_range_edge_label(energy.min(), energy.max()))
print('\nAutomatically determined absorption edge:')
print('\n'.join(f"{en} eV : {lab}" for en, lab in available_l_edges))
print(f"Edge label: {edge_label}")

In [ ]:
# Load scan as Spectra object
xas_scan = scan.xas_scan()
scan_title = f"#{xas_scan.metadata.scan_no} {edge_label} T={xas_scan.metadata.temp:.0f}K B={xas_scan.metadata.mag_field:.3g}T {xas_scan.metadata.pol}"
print(scan_title)
print(xas_scan)

In [ ]:
# Plot raw spectra
fig, axes = plt.subplots(1, 2, figsize=(12, 4), dpi=100)
fig.suptitle(scan_title)
for n, (mode, spectra) in enumerate(xas_scan.spectra.items()):
    spectra.plot(ax=axes[n], label=xas_scan.name)
    axes[n].set_ylabel(mode)
    for lab, en in available_l_edges:
        axes[n].axvline(en, c='k', linestyle='-')
        axes[n].text(en+1, 0.98 * spectra.signal.max(), lab)

for ax in axes.flat:
    ax.set_xlabel('E [eV]')
    ax.legend()

In [ ]:
# 1. Normalise by pre-edge
xas_scan.divide_by_preedge()

# plot scan normalised scan files
fig, axes = plt.subplots(1, 2, figsize=(12, 4), dpi=100)
fig.suptitle('Normalise by pre-edge')
for n, (mode, spectra) in enumerate(xas_scan.spectra.items()):
    spectra.plot(ax=axes[n], label=xas_scan.name)
    axes[n].set_ylabel(mode)

for ax in axes.flat:
    ax.set_xlabel('E [eV]')
    ax.legend()


In [ ]:
# # 2. Fit and subtract background
# xas_scan.auto_edge_background(peak_width_ev=10.)
#
# # Plot background subtracted scans
# fig, axes = plt.subplots(2, 2, figsize=(12, 10), dpi=80)
# fig.suptitle(scan_title)
# for n, (mode, spectra) in enumerate(xas_scan.spectra.items()):
#     spectra.plot_parents(ax=axes[0, n])
#     spectra.plot_bkg(ax=axes[0, n])
#     axes[0, n].set_ylabel(mode)
#
#     spectra.plot(ax=axes[1, n], label=xas_scan.name)
#     axes[1, n].set_ylabel(mode)
#
# for ax in axes.flat:
#     ax.set_xlabel('E [eV]')
#     ax.legend()

In [ ]:
# 4. Save Nexus file
xas_scan.write_nexus(outpath)

# notify GDA-Datavis that the file exists
# (this just sends a message, the GDA-DataVis perspective must be open for this to appear)
try:
    from mmg_toolbox.utils.gda_functions import gda_datavis_file_message
    gda_datavis_file_message(outpath)
except ImportError:
    print('Not running on beamline!')

# Search for previous NXxas scans
If the previous scan was also a spectra scan, plot this and take the difference

In [ ]:
# check previous ~10 scans and build a list of energy scans at the same edge, temperature and field
# see help(find_similar_measurements) for more info
scans = xas.find_similar_measurements(inpath)
pols = {s.metadata.pol for s in scans}

print(f"Unique polarisations: {pols}")

if len(pols) > 1:
    for xas_scan in scans:
        m = xas_scan.metadata
        print(f"{m.scan_no} T={m.temp:3.0f}K, B={m.mag_field:4.1f}T,  {m.pol}")
else:
    print('Not enough polarisations')

In [ ]:
# Repeat steps 1-3 for each scan
if len(pols) > 1:
    for xas_scan in scans:
        xas_scan.divide_by_preedge()
        print(xas_scan)

    # plot scan normalised scan files
    fig, axes = plt.subplots(1, 2, figsize=(12, 4), dpi=100)
    fig.suptitle('Normalise by pre-edge')
    for xas_scan in scans:
        for n, (mode, spectra) in enumerate(xas_scan.spectra.items()):
            spectra.plot(ax=axes[n], label=f"{xas_scan.name} {xas_scan.metadata.pol}")
            axes[n].set_ylabel(mode)

    for ax in axes.flat:
        ax.set_xlabel('E [eV]')
        ax.legend()
else:
    print('Not enough polarisations')


In [ ]:
# Fit and subtract background
if len(pols) > 1:
    # for scan in scans:
    #     scan.auto_edge_background(peak_width_ev=10.)

    # Plot background subtracted scans
    for xas_scan in scans:
        fig, axes = plt.subplots(2, 2, figsize=(16, 10), dpi=80)
        scan_title = f"#{xas_scan.metadata.scan_no} {edge_label} T={xas_scan.metadata.temp:.0f}K B={xas_scan.metadata.mag_field:.3g}T {xas_scan.metadata.pol}"
        fig.suptitle(scan_title)
        for n, (mode, spectra) in enumerate(xas_scan.spectra.items()):
            spectra.plot_parents(ax=axes[0, n])
            spectra.plot_bkg(ax=axes[0, n])
            axes[0, n].set_ylabel(mode)

            spectra.plot(ax=axes[1, n], label=xas_scan.name)
            axes[1, n].set_ylabel(mode)

        for ax in axes.flat:
            ax.set_xlabel('E [eV]')
            ax.legend()


In [ ]:
if len(pols) > 1:
    from mmg_toolbox.xas.spectra_container import average_polarised_scans

    # Average polarised scans
    for xas_scan in scans:
        print(f"{xas_scan.name}: {xas_scan.metadata.pol}")
    pol1, pol2 = average_polarised_scans(*scans)
    print(pol1)
    print(pol2)

    # Plot averaged scans
    fig, axes = plt.subplots(1, 2, figsize=(12, 4), dpi=100)
    fig.suptitle('Averaged polarised scans')
    for xas_scan in [pol1, pol2]:
        for n, (mode, spectra) in enumerate(xas_scan.spectra.items()):
            spectra.plot(ax=axes[n], label=xas_scan.name)
            axes[n].set_ylabel(mode)

    for ax in axes.flat:
        ax.set_xlabel('E [eV]')
        ax.legend()


# Calculate XMCD/XMLD

In [ ]:
if len(pols) > 1:
    # Calculate XMCD
    xmcd = pol1 - pol2
    print(xmcd)

    for name, spectra in xmcd.spectra.items():
        print(spectra)
        print(spectra.process)
        print(spectra.sum_rules_report(1))


In [ ]:
if len(pols) > 1:
    fig, axes = plt.subplots(1, 2, figsize=(12, 4), dpi=100)
    fig.suptitle(xmcd.name.upper())
    for n, (mode, spectra) in enumerate(xmcd.spectra.items()):
        spectra.plot(ax=axes[n])
        axes[n].set_ylabel(mode)

    for ax in axes.flat:
        ax.set_xlabel('E [eV]')
        ax.legend()

In [ ]:
if len(pols) > 1:
    # Save xmcd file
    xmcd_filename = f"{scans[0].metadata.scan_no}-{scans[-1].metadata.scan_no}_{xmcd.name}.nxs"
    output_folder = os.path.dirname(outpath)
    xmcd_filepath = os.path.join(output_folder, xmcd_filename)
    xmcd.write_nexus(xmcd_filepath)

    # notify GDA-Datavis that the file exists
    # (this just sends a message, the GDA-DataVis perspective must be open for this to appear)
    try:
        from mmg_toolbox.utils.gda_functions import gda_datavis_file_message
        gda_datavis_file_message(xmcd_filepath)
    except ImportError:
        print('Not running on beamline!')